## Autograd

In [1]:
from typing import Union, List

import numpy as np
from numpy import ndarray as array

In [2]:
a = 3
a.__add__(4)

7

In [3]:
np.array(3)

array(3)

In [4]:
c = array(3)
c

array([5.94842666e-091, 5.49604184e+170, 1.56088177e+184])

In [5]:
a = array([3,3])
print("Addition using '__add__':", a.__add__(4))
print("Addition using '+':", a + 4)

Addition using '__add__': [[2.68156159e+154 2.32035264e+077 4.00000000e+000]
 [4.00000000e+000 4.00000000e+000 4.00000000e+000]
 [4.00000000e+000 4.00000000e+000 4.00000000e+000]]
Addition using '+': [[2.68156159e+154 2.32035264e+077 4.00000000e+000]
 [4.00000000e+000 4.00000000e+000 4.00000000e+000]
 [4.00000000e+000 4.00000000e+000 4.00000000e+000]]


In [6]:
Addable = Union[float, int]

Numberable = Union[Addable, float, int]

def ensure_Number(num: Numberable):
    if isinstance(num, NumberWithGrad):
        return num
    else:
        return NumberWithGrad(num)        

class NumberWithGrad(object):
    
    def __init__(self, 
                 num: Addable,
                 depends_on: List[Addable] = None,
                 creation_op: str = ''):
        self.num = num
        self.grad = None
        self.depends_on = depends_on or []
        self.creation_op = creation_op

    def __add__(self, 
                other: Numberable):
        return NumberWithGrad(self.num + ensure_Number(other).num,
                              depends_on = [self, ensure_Number(other)],
                              creation_op = 'add')
    
    def __mul__(self,
                other: Numberable = None):

        return NumberWithGrad(self.num * ensure_Number(other).num,
                              depends_on = [self, ensure_Number(other)],
                              creation_op = 'mul')
    def sum(self):

        return special_sum(self)
    
    def backward(self, backward_grad: Addable = None):
        if backward_grad is None: # first time calling backward
            self.grad = 1
        else: 
            if self.grad is None:
                self.grad = backward_grad
            else:
                self.grad += backward_grad
        
        if(self.creation_op == "add"):
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad)    

        if(self.creation_op == "mul"):
            
            new = self.depends_on[1] * self.grad
            self.depends_on[0].backward(new.num)
            new = self.depends_on[0] * self.grad
            self.depends_on[1].backward(new.num)

In [7]:
a = NumberWithGrad(3)

In [8]:
b = a * 4
c = b + 3
d = (a + 2)
e = c * d 
e.backward()

In [9]:
a.grad

35

## PyTorch

In [10]:
import torch
from torch import Tensor

In [11]:
torch.manual_seed(20190324)
array_data = np.full((2,2), 3.0)
print(array_data)
a = torch.tensor(array_data, requires_grad=True)
print(a)

[[3. 3.]
 [3. 3.]]
tensor([[3., 3.],
        [3., 3.]], dtype=torch.float64, requires_grad=True)


In [12]:
b = a * 4
c = b + 3
d = (a + 2)
e = c * d
e_sum = e.sum()
e_sum.backward()

In [13]:
a.grad

tensor([[35., 35.],
        [35., 35.]], dtype=torch.float64)